In [ ]:
import os
import sys
import traceback
import datetime
import tempfile

In [ ]:
RESOLVE_SCRIPT_API = r"%PROGRAMDATA%\Blackmagic Design\DaVinci Resolve\Support\Developer\Scripting"
RESOLVE_SCRIPT_LIB = r"C:\Program Files\Blackmagic Design\DaVinci Resolve\fusionscript.dll"
PYTHONPATH = r"%RESOLVE_SCRIPT_API%\Modules\\"

os.environ['RESOLVE_SCRIPT_API'] = RESOLVE_SCRIPT_API
while '%' in PYTHONPATH:
    PYTHONPATH = os.path.expandvars(PYTHONPATH)
if PYTHONPATH not in sys.path:
    sys.path.append(PYTHONPATH)

In [ ]:
import DaVinciResolveScript as dvr

resolve = dvr.scriptapp("Resolve")
fusion = resolve.Fusion()
ui = fusion.UIManager
disp = dvr.UIDispatcher(ui)

projectManager = resolve.GetProjectManager()
project = projectManager.GetCurrentProject()
mediapool = project.GetMediaPool()
# framerate = project.GetSetting('timelineFrameRate')
timeline = project.GetCurrentTimeline()
framerate = int(timeline.GetSetting("timelineFrameRate"))
startframe = timeline.GetStartFrame()

In [ ]:
# timeline.AddTrack('video')
# trackindex = timeline.GetTrackCount('video')
# timeline.DeleteTrack('video', timeline.GetTrackCount('video'))

In [ ]:
timeline.AddTrack('subtitle')
trackindex = timeline.GetTrackCount('subtitle')

In [ ]:
folder = mediapool.GetCurrentFolder()
folder.GetClipList()
mediaPoolItem = None
for item in folder.GetClipList():
    # print(item.GetName())
    # print(item.GetClipProperty().get("Clip Name", None))
    if item.GetClipProperty().get("Clip Name", None) == 'Subtitle':
        mediaPoolItem = item

In [ ]:
trackindex

In [ ]:
newClip = {
    "mediaPoolItem": mediaPoolItem,
    "startFrame": 0,
    "endFrame":   10,
    "trackIndex": trackindex,
    "recordFrame": 0,
}
mediapool.AppendToTimeline()

In [ ]:
timeline.DeleteTrack('video', timeline.GetTrackCount('video'))

In [ ]:
folder = mediapool.GetCurrentFolder()
folder.GetClipList()
mediaPoolItem = None
for item in folder.GetClipList():
    # print(item.GetName())
    if item.GetClipProperty().get("Clip Name", None) == 'TextTemplate':
        mediaPoolItem = item

subitems = timeline.GetItemListInTrack('subtitle', 1)
for sub in subitems:
    newClip = {
        "mediaPoolItem": mediaPoolItem,
        "startFrame": 0,
        "endFrame":  sub.GetEnd() -  sub.GetStart(),
        "trackIndex": trackindex,
        "recordFrame": sub.GetStart(),
    }
    mediapool.AppendToTimeline([newClip])

clipList = timeline.GetItemListInTrack('video', trackindex)

for idx, clip in enumerate(clipList):
    clip.SetClipColor('Orange')

    comp = clip.GetFusionCompByIndex(1)
    if comp:
        toollist = list(comp.GetToolList().values())
        for tool in toollist:
            if tool.GetAttrs()["TOOLS_Name"] == 'Template':
                comp.SetActiveTool(tool)
                tool.SetInput('StyledText', subitems[idx].GetName())
    clip.SetClipColor('Teal')